<a href="https://colab.research.google.com/github/koojayeong/TextAnalysis/blob/main/Bag_of_Words_Meets_Bags_of_Popcorn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# install kaggle 
!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle competitions list

     |████████████████████████████████| 58 kB 2.9 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=530b49129a874dfc97b76d27053472fffa6a665b84e2e6882770b4a2aa41e651
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
cd /content/drive/MyDrive/kaggle/BagsOfPopcorn

/content/drive/MyDrive/kaggle/BagsOfPopcorn


In [ ]:
!kaggle competitions download -c word2vec-nlp-tutorial

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
# !unzip word2vec-nlp-tutorial.zip

Archive:  word2vec-nlp-tutorial.zip
replace labeledTrainData.tsv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace testData.tsv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace unlabeledTrainData.tsv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
# !unzip labeledTrainData.tsv.zip
# !unzip unlabeledTrainData.tsv.zip
# !unzip testData.tsv.zip

Archive:  labeledTrainData.tsv.zip
replace labeledTrainData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  unlabeledTrainData.tsv.zip
replace unlabeledTrainData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  testData.tsv.zip
replace testData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


# ---------Part 1---------

# Reading the data

In [ ]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/labeledTrainData.tsv.zip", 
                    header=0, # the first line of the file contains column names
                    delimiter="\t", # the fields are separated by tabs
                    quoting=3) # ignore doubled quotes

In [ ]:
train.shape

(25000, 3)

In [ ]:
train.columns.values

array(['id', 'sentiment', 'review'], dtype=object)

In [ ]:
#@title 기본 제목 텍스트
print( train["review"][0] )

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

# Data Cleaning and Text Preprocessing

Removing HTML Markup: The BeautifulSoup Package

In [ ]:
pip install BeautifulSoup4

In [ ]:
from bs4 import BeautifulSoup

example1 = BeautifulSoup(train["review"][0])

print( train["review"][0] )
print( example1.get_text())

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

Dealing with Punctuation, Numbers and Stopwords: NLTK and regular expressions  
  
 * "!!!" or ":-(" 같은 것들은 감정을 담은 words로 취급되어야 한다.  
 * 숫자들은 words로 취급하거나 "NUM" 으로 대체
 *  punctuation and numbers 을 제거할 땐 re 패키지 사용

In [ ]:
import re
letters_only = re.sub("[^a-zA-Z]",
                      " ",
                      example1.get_text())
print(letters_only)

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

 tokenization :   
글자들을 소문자로 바꾸고 개별 단어로 split 하기

In [ ]:
lower_case = letters_only.lower()
words = lower_case.split()

stopwords :  
큰 의미 없고 자주 등장하는 단어들을 말한다.  
영어에서는 a, is, the 와 같은 조사들  
NLTK 사용

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
words = [w for w in words if not w in stopwords.words("english")]
print(words)

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

u'towards', u'press', u'also' 같은 문자열 앞 "u"는 unicode string으로서 나타내는 것을 말한다.

Porter Stemming, Lemmatizing 등 다르게 표현된 같은 단어들을 처리하는 방법이 있다.  
"messages", "message", "messaging"

In [ ]:
def review_to_words( raw_review) :
  """
  raw review를 string of words 로 바꿔주기 위한 함수
  """
  # 1. remove html
  review_text = BeautifulSoup(raw_review).get_text()

  # 2. remove non-letters
  letters_only = re.sub("[^a-zA-Z]", 
                        " ",
                        review_text)
  
  # 3. convert to lower case, split into individual words
  words = letters_only.lower().split()

  # 4. python. searching a set is much faster than searching
  # a list, so convert the stop words to a set
  stops = set(stopwords.words("english"))

  # 5. remove stop words
  meaningful_words = [w for w in words if not w in stops]

  # 6. one string 으로 반환
  return " ".join(meaningful_words)

data type 으로 set을 이용하면 속도가 빠르다. 

In [ ]:
clean_review = review_to_words(train["review"][0])
print(clean_review)

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [ ]:
# 진행 상황 확인

print("Cleaning and parsing the training set movie reviews...\n")

num_reviews = train["review"].size

clean_train_reviews = []

for i in range( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_reviews ))                                                                    
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



# Creating Features from a Bag of Words(using scikit-learn)

IMDB 데이터를 bag of word 표현법으로 할 것이다. 가장 빈도수가 높은 5000개의 단어를 활용할 것이다. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000)

train_data_features = vectorizer.fit_transform(clean_train_reviews)

train_data_features = train_data_features.toarray()

In [ ]:
print(train_data_features.shape)

(25000, 5000)


In [ ]:
vocab = vectorizer.get_feature_names()
print(vocab)

['abandoned', 'abc', 'abilities', 'ability', 'able', 'abraham', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accompanied', 'accomplished', 'according', 'account', 'accuracy', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'activities', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'ad', 'adam', 'adams', 'adaptation', 'adaptations', 'adapted', 'add', 'added', 'adding', 'addition', 'adds', 'adequate', 'admire', 'admit', 'admittedly', 'adorable', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventure', 'adventures', 'advertising', 'advice', 'advise', 'affair', 'affect', 'affected', 'afford', 'aforementioned', 'afraid', 'africa', 'african', 'afternoon', 'afterwards', 'age', 'aged', 'agent', 'agents', 'ages', 'aging', 'ago', 'ag

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
import numpy as np

dist = np.sum(train_data_features, axis=0)

for tag, count in zip(vocab, dist) :
  print(count, tag)

187 abandoned
125 abc
108 abilities
454 ability
1259 able
85 abraham
116 absence
83 absent
352 absolute
1485 absolutely
306 absurd
192 abuse
91 abusive
98 abysmal
297 academy
485 accent
203 accents
300 accept
130 acceptable
144 accepted
92 access
318 accident
200 accidentally
88 accompanied
124 accomplished
296 according
186 account
81 accuracy
284 accurate
123 accused
179 achieve
139 achieved
124 achievement
90 acid
971 across
1251 act
658 acted
6490 acting
3354 action
311 actions
83 activities
2389 actor
4486 actors
1219 actress
369 actresses
394 acts
793 actual
4237 actually
148 ad
302 adam
98 adams
453 adaptation
80 adaptations
154 adapted
810 add
439 added
166 adding
347 addition
337 adds
113 adequate
124 admire
621 admit
134 admittedly
101 adorable
510 adult
376 adults
100 advance
90 advanced
153 advantage
510 adventure
204 adventures
91 advertising
259 advice
90 advise
346 affair
93 affect
113 affected
104 afford
126 aforementioned
343 afraid
212 africa
255 african
187 afternoon

# Random Forest

In [ ]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators= 100)

forest = forest.fit(train_data_features, train["sentiment"])

Training the random forest...


In [ ]:
test = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/testData.tsv",
                   header = 0,
                   delimiter= '\t',
                   quoting = 3)

print(test.shape)

num_reviews = len(test["review"])
clean_test_reviews = []

print("Cleaning and parsing the test set movie reviews...\n")

for i in range(0, num_reviews):
    if( (i+1) % 1000 == 0 ):
        print("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

result = forest.predict(test_data_features)

output = pd.DataFrame(data = {"id": test["id"], "sentiment": result})

output.to_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/Bag_of_Words_model.csv",
              index=False,
              quoting=3)

(25000, 2)
Cleaning and parsing the test set movie reviews...

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [ ]:
# kaggle api 로 제출하기
# !kaggle competitions submit -c word2vec-nlp-tutorial -f /content/drive/MyDrive/kaggle/BagsOfPopcorn/Bag_of_Words_model.csv -m "21.12.28"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


# ---------Part 2---------

# Word2Vec : Distributed Word Vectors

gensim 패키지를 이용해 Word2Vec 구현 사용.   
  
word2vec은 컴퓨팅 집약적이므로 멀티스레딩에 의존한다. cython을 설치해야 적절한 시간 내에 모델 훈련할 수 있다.

In [ ]:
import sys
import cython

import numba
from numba import jit

import numpy as np

In [ ]:
pip install gensim

# Preparing to Train a Model

In [ ]:
import pandas as pd

# read data from files
train = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/labeledTrainData.tsv",
                    header=0,
                    delimiter='\t',
                    quoting=3)
test = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/testData.tsv",
                    header=0,
                    delimiter='\t',
                    quoting=3)
unlabeled_train = pd.read_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/unlabeledTrainData.tsv",
                    header=0,
                    delimiter='\t',
                    quoting=3)

print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))


Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [ ]:
# string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False) :
  # 1. remove html
  review_text = BeautifulSoup(review).get_text()

  # 2. remove non-letters
  review_text = re.sub("[^a-zA-Z]", " ", review_text)

  # 3. 소문자화, split
  words = review_text.lower().split()

  # 4. 선택사항 : stopwords
  if remove_stopwords :
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]

  return words

word2voec 의 입력 형식은 list의 lists 이다. 문단을 문장으로 분리해야 하는데 "?", "!", "." 으로 끝나거나 공백, 대문자 등 명확한 하나의 기준은 없다.  
   
문장 분할을 위해 nltk의 토큰나이저를 사용할 것이다.

In [ ]:
import nltk
nltk.download('punkt')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_sentences(review, tokenizer, remove_stopwords=False) :
  """
  review를 parased sentences로 split한다.
  list of sentences 를 반환한다.
  """
  # 1. use nltk tokenizer to split paragraph into sentences
  raw_sentences = tokenizer.tokenize(review.strip())

  # 2. loop over each sentence
  sentences = []
  for raw_sentence in raw_sentences :
    # 만약 sentence 가 비었다면 skip it.
    if len(raw_sentence) > 0 :
      sentences.append(review_to_wordlist(raw_sentence,
                                          remove_stopwords))
  return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sentences = []

print("Parsing sentences from training set")
for review in train["review"]:
  sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
  sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

In [ ]:
print(len(sentences))

print(sentences[0])

print(sentences[1])

795538
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


# Training and Saving Model

* Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.
* Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
* Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
* Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.
* Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
* Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
* Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

In [ ]:
# # output 메세지 format
# import logging 
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
#                     level = logging.INFO)

# # parameter 값 setting
# num_features = 300    # Word vector dimensionality                      
# min_word_count = 40   # Minimum word count                        
# num_workers = 4       # Number of threads to run in parallel
# context = 10          # Context window size                                                                                    
# downsampling = 1e-3   # Downsample setting for frequent words

# # model 초기화, train
# from gensim.models import word2vec
# print("Training moel...")
# model = word2vec.Word2Vec(sentences,
#                           workers=num_workers,
#                           size=num_features,
#                           min_count=min_word_count,
#                           window=context,
#                           sample=downsampling)

# # model이 더 학습되지 않는다면 calling
# # init_sims가 모델을 더 memory 효율적으로 만든다
# model.init_sims(replace=True)

# model_name="300features_40minwords_10context"
# model.save(model_name)

2021-12-28 22:13:16,083 : INFO : 'pattern' package not found; tag filters are not available for English
2021-12-28 22:13:16,095 : INFO : collecting all words and their counts
2021-12-28 22:13:16,096 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-12-28 22:13:16,168 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2021-12-28 22:13:16,230 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2021-12-28 22:13:16,295 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training moel...


2021-12-28 22:13:16,361 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2021-12-28 22:13:16,428 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2021-12-28 22:13:16,496 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2021-12-28 22:13:16,564 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2021-12-28 22:13:16,631 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2021-12-28 22:13:16,699 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2021-12-28 22:13:16,763 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2021-12-28 22:13:16,826 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2021-12-28 22:13:16,891 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

In [ ]:
# directory = "/content/drive/MyDrive/kaggle/BagsOfPopcorn/"
# model_name="300features_40minwords_10context"
# model.save(directory+model_name)

2021-12-28 22:16:50,610 : INFO : saving Word2Vec object under /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context, separately None
2021-12-28 22:16:50,618 : INFO : not storing attribute vectors_norm
2021-12-28 22:16:50,623 : INFO : not storing attribute cum_table
2021-12-28 22:16:51,170 : INFO : saved /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context


# Exploring the Model Results

"doesn_match" 함수는 집합에서 어떤 단어가 다른 단어와 가장 다른지 추론한다.

In [ ]:
from gensim.models import Word2Vec

directory = "/content/drive/MyDrive/kaggle/BagsOfPopcorn/"
model_name="300features_40minwords_10context"

model = Word2Vec.load(directory+model_name)

In [ ]:
model.doesnt_match("man woman child kitchen".split())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'kitchen'

In [ ]:
model.doesnt_match("france england germany berlin".split())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'berlin'

In [ ]:
model.doesnt_match("paris berlin london austria".split())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'paris'

또한 "most_simply" 함수를 사용하여 모형의 단어 군집에 대한 통찰력을 얻을 수 있습니다.

In [ ]:
model.most_similar("man")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('woman', 0.6156646013259888),
 ('lady', 0.5828548669815063),
 ('lad', 0.5588903427124023),
 ('guy', 0.5390889644622803),
 ('soldier', 0.5319414138793945),
 ('person', 0.5242750644683838),
 ('men', 0.5128154754638672),
 ('businessman', 0.5077592134475708),
 ('millionaire', 0.49865302443504333),
 ('monk', 0.49851858615875244)]

In [ ]:
model.most_similar("queen")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('princess', 0.6786762475967407),
 ('eva', 0.6233296990394592),
 ('maid', 0.5981199741363525),
 ('victoria', 0.5915231704711914),
 ('goddess', 0.5868721008300781),
 ('mistress', 0.5843722820281982),
 ('stepmother', 0.5828476548194885),
 ('bride', 0.5771623849868774),
 ('showgirl', 0.5746123194694519),
 ('maria', 0.5695031881332397)]

In [ ]:
model.most_similar("awful")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('terrible', 0.7761105895042419),
 ('horrible', 0.7338160276412964),
 ('atrocious', 0.7311441898345947),
 ('dreadful', 0.7049832344055176),
 ('abysmal', 0.7036162614822388),
 ('horrid', 0.684990644454956),
 ('appalling', 0.679607093334198),
 ('horrendous', 0.6790891885757446),
 ('lousy', 0.6275255680084229),
 ('amateurish', 0.6103198528289795)]

# ---------Part 3---------

# Numeric Representations of Words

trained model에서 단어의 의미적인 이해를 하기 위해 무엇을 사용해야 할까?   
"syn0" 이라 불리는 numpy에 vocabulary의 각 단어의 feature vector가 저장되어 있다. 

KeyedVectors.load_word2vec_format instead of  Word2Vec.load_word2vec_format
  
word2vec_model.wv.save_word2vec_format instead of  word2vec_model.save_word2vec_format
  
model.wv.syn0norm instead of  model.syn0norm
  
model.wv.syn0 instead of  model.syn0
  
model.wv.vocab instead of model.vocab
  
model.wv.index2word instead of  model.index2word
  

In [ ]:
from gensim.models import Word2Vec

directory = "/content/drive/MyDrive/kaggle/BagsOfPopcorn/"
model_name="300features_40minwords_10context"

model = Word2Vec.load(directory+model_name)

2021-12-28 22:17:59,806 : INFO : loading Word2Vec object from /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context
2021-12-28 22:18:00,475 : INFO : loading wv recursively from /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context.wv.* with mmap=None
2021-12-28 22:18:00,477 : INFO : setting ignored attribute vectors_norm to None
2021-12-28 22:18:00,482 : INFO : loading vocabulary recursively from /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context.vocabulary.* with mmap=None
2021-12-28 22:18:00,485 : INFO : loading trainables recursively from /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context.trainables.* with mmap=None
2021-12-28 22:18:00,489 : INFO : setting ignored attribute cum_table to None
2021-12-28 22:18:00,492 : INFO : loaded /content/drive/MyDrive/kaggle/BagsOfPopcorn/300features_40minwords_10context


model의 wv.vectors에 임베딩 벡터가 저장되어 있다.  
numpy 배열에 저장된 어휘의 각 단어에 대한 특징 벡터로 구성되어 있다.

In [ ]:
# embedding vector
wv = model.wv.vectors

print(type(wv))
print(wv.shape)

<class 'numpy.ndarray'>
(16490, 300)


In [ ]:
# idx to count
vocabs = model.wv.vocab

print(type(vocabs))

<class 'dict'>


임베딩 벡터의 행 수는 모델의 어휘에 있는 단어의 수이며, 열의 수는 파트 2에서 설정한 피처 벡터의 크기와 일치한다. 최소 단어 수를 40개로 설정하면 각각 300개씩 총 16,492개의 단어가 어휘로 제공되었습니다. 개별 단어 벡터는 다음과 같은 방법으로 접근할 수 있다.

In [ ]:
model["flower"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.09921692,  0.02653319, -0.02807025,  0.03648569, -0.0858695 ,
        0.06671294, -0.08105352, -0.02811643, -0.021209  , -0.03963399,
        0.0125827 , -0.07173354,  0.03243794, -0.02927999, -0.03764388,
       -0.01172583,  0.0278274 , -0.1083589 , -0.10078763, -0.00118993,
        0.01049133, -0.12069897, -0.05284834,  0.00325722, -0.09335221,
       -0.06897346, -0.00463652,  0.02599252,  0.03755717, -0.03023862,
       -0.03936527, -0.03897588,  0.03495418, -0.00989921, -0.06467859,
        0.04313026, -0.04321264,  0.00943176,  0.07160495,  0.00069874,
       -0.03811093,  0.03163365, -0.09359701,  0.10098181, -0.01307055,
        0.02385334,  0.01754909, -0.00151221,  0.00499224, -0.00744105,
        0.08824725, -0.02470805,  0.03512256, -0.03333938,  0.04032306,
        0.07827656,  0.06674298,  0.11852133, -0.07978975, -0.02442353,
        0.00513156, -0.03342777, -0.00485153, -0.03865312, -0.09084394,
        0.03436586,  0.01470653,  0.07579441, -0.07238955,  0.01

# From Words To Paragraphs, Attempt 1 : Vector Averaging
특징 벡터 평균

IMDB 데이터 세트의 한 문제는 review의 길이가 다양하다는 것이다. 우리는 개별 단어 벡터를 취해서 개별단어벡터를 모든 리뷰에 같은 길이의 특징 세트로 변환할 방법이 필요하다.  
  
단어 벡터는 300-차원  
  
방법1은 단어벡터를 주어진 리뷰에 평균내는 것. 이걸 위해서 stop words를 제거했음  


In [90]:
import numpy as np

def makeFeatureVec(words, model, num_features) :
  """
  주어진 문단에 단어 벡터 평균내기
  """
  # speed를 위해서 빈 numpy array 초기화
  featureVec = np.zeros((num_features,), dtype="float32")
  
  nwords = 0.

  # index2word 는 model의 vocabulary의 단어들을 가지고 있다.
  # speed를 위해 set으로 바꿔주기
  index2word_set = set(model.wv.index2word)

  # 리뷰의 각 단어에 반복문
  for word in words:
    if word in index2word_set :
      nwords += 1
      featureVec = np.add(featureVec, model[word])
  
  # 평균내기
  featureVec = np.divide(featureVec,nwords)
  
  return featureVec

def getAvgFeatureVecs(reviews, model, num_features) :
  """
  revies set 주어지면 feature vector 평균 계산 후
  2d numpy array 반환
  """
  # counter 초기화하기
  counter = 0

  # speed를 위해 2d numpy array 미리 할당
  reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")

  # 리뷰마다 반복문
  for review in reviews:
    
    # 1000 마다 상태 메세지 출력
    if counter%1000 == 0 :
      print("Review %d of %d"%(counter, len(reviews)))

    # feature vector 평균 함수 사용
    reviewFeatureVecs[counter] = makeFeatureVec(review, 
                                                model,
                                                num_features)
    
    counter += 1
  return reviewFeatureVecs

In [92]:
# 정의한 함수들 불러와서 사용
# training, testing sets 를 평균 feature vectors 계산

num_features = 300 

print("Creating average feature vecs for train reviews")
clean_train_reviews = []
for review in train["review"] :
  clean_train_reviews.append( review_to_wordlist(review,
                                                 remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, 
                                  model,
                                  num_features)

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
  clean_test_reviews.append(review_to_wordlist(review,
                                               remove_stopwords=True))

testDataVecs = getAvgFeatureVecs(clean_test_reviews,
                                 model,
                                 num_features)

Creating average feature vecs for train reviews
Review 0 of 25000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 

In [95]:
# pickle 로 저장
import pickle

with open("/content/drive/MyDrive/kaggle/BagsOfPopcorn/trainDataVecs","wb") as fw:
  pickle.dump(trainDataVecs, fw)

with open("/content/drive/MyDrive/kaggle/BagsOfPopcorn/testDataVecs","wb") as fw:
  pickle.dump(testDataVecs, fw)

In [98]:
# pickle 불러오기
import pickle

with open("/content/drive/MyDrive/kaggle/BagsOfPopcorn/trainDataVecs","rb") as fr:
  trainDataVecs = pickle.load(fr)

with open("/content/drive/MyDrive/kaggle/BagsOfPopcorn/testDataVecs","rb") as fr:
  testDataVecs = pickle.load(fr)

평균 단락 벡터를 사용하여 random forest를 훈련시킨다. 

In [96]:
# 100 tree 사용해서 random forest 훈련
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)

print("Fitting a random forest to labeled training data")
forest = forest.fit(trainDataVecs, train["sentiment"])

# Text & extract results
result = forest.predict(testDataVecs)

# Write the test results
output = pd.DataFrame(data={"id":test["id"], "sentiment": result})
output.to_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/Word2Vec_AverageVectors.csv", 
              index=False, 
              quoting=3)

Fitting a random forest to labeled training data


성능을 높이기 위해  "tf-idf" 가중치를 사용할 수 있다.

# From Words to Paragraphs, Attempt 2 : Clustering

Word2Vec는 의미론적으로 관련된 단어의 클러스터를 생성하므로 가능한 다른 접근 방식은 클러스터 내에서 단어의 유사성을 이용하는 것이다. 이러한 방식으로 벡터를 그룹화하는 것을 벡터 양자화라고 한다. 이를 위해서는 먼저 K-평균과 같은 클러스터링 알고리즘을 사용하여 할 수 있는 단어 클러스터의 중심을 찾아야 한다.

K-평균에서 설정해야 할 모수는 "K" 즉 군집 수입니다. 생성할 클러스터 수를 어떻게 결정해야 합니까? 시행착오는 군집당 평균 5개 정도의 단어만을 가진 작은 군집이 단어가 많은 큰 군집보다 더 나은 결과를 낸다는 것을 시사했다. 클러스터링 코드는 아래에 제시되어 있습니다. 우리는 K-평균을 수행하기 위해 skit-learn을 사용한다.

큰 K를 사용한 K-평균 클러스터링은 매우 느릴 수 있습니다. 다음 코드는 컴퓨터에서 40분 이상 걸렸습니다. 아래는 K-평균 함수에 타이머를 설정하여 시간이 얼마나 걸리는지 알아봅니다.

In [101]:
from sklearn.cluster import KMeans
import time

start = time.time()

# "k"(num_clusters) 를 vocabulary size의 1/5
# 혹은 클러스터당 5개 단어의 평균
word_vectors = model.wv.vectors
num_clusters = word_vectors.shape[0]//5

# k-means 객체를 초기화, centroids 추출하는데 사용
kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

end = time.time()
elapsed = end-start
print("Time taken for k Means clustering", elapsed, "seconds.")


Time taken for k Means clustering 1381.4521040916443 seconds.


In [102]:
# pickle 로 저장
import pickle

with open("/content/drive/MyDrive/kaggle/BagsOfPopcorn/kmeans_clustering_idx","wb") as fw:
  pickle.dump(idx, fw)

In [116]:
# word/index dictionary
# 각 단어를 cluster number에 매핑
word_centroid_map = dict(zip(model.wv.index2word, idx))

In [119]:
print(word_centroid_map.keys())
print(word_centroid_map.values())

dict_keys(['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'with', 'for', 'movie', 'but', 'film', 'you', 't', 'on', 'not', 'he', 'are', 'his', 'have', 'be', 'one', 'all', 'at', 'they', 'by', 'who', 'an', 'from', 'so', 'like', 'there', 'her', 'or', 'just', 'about', 'out', 'has', 'if', 'what', 'some', 'good', 'can', 'more', 'when', 'very', 'she', 'up', 'no', 'time', 'even', 'would', 'my', 'which', 'their', 'story', 'only', 'really', 'see', 'had', 'were', 'well', 'we', 'me', 'than', 'much', 'bad', 'get', 'been', 'people', 'also', 'into', 'do', 'great', 'other', 'will', 'first', 'because', 'him', 'how', 'most', 'don', 'them', 'made', 'its', 'make', 'then', 'way', 'could', 'too', 'movies', 'after', 'any', 'characters', 'character', 'think', 'films', 'two', 'watch', 'being', 'many', 'plot', 'seen', 'never', 'where', 'love', 'life', 'little', 'acting', 'best', 'did', 'over', 'off', 'know', 'show', 'ever', 'does', 'man', 'better', 'your', 'here', 'end', 

In [120]:
for cluster in range(0,10) :

  # cluster number print
  print("\nCluster %d"%cluster)
  values = list (word_centroid_map.values())
  keys = list(word_centroid_map.keys())
  # cluster number에 대한 모든 단어들을 찾고 출력하기
  words = []
  for i in range(0, len(values)):
    if values[i]==cluster:
      words.append(keys[i])
  print(words)


Cluster 0
['scriptwriting']

Cluster 1
['dern', 'debra', 'reese', 'nana', 'witherspoon', 'leachman', 'winger', 'moonstruck', 'cloris', 'cuthbert', 'breslin', 'kher']

Cluster 2
['detective', 'agent', 'officer', 'attorney', 'undercover', 'convict', 'investigator', 'alibi', 'investigative', 'defender']

Cluster 3
['countries', 'groups', 'areas', 'cities', 'peoples', 'nations', 'tribes']

Cluster 4
['waving', 'punching', 'mercilessly', 'goo', 'spewing', 'drooling', 'spraying', 'feces']

Cluster 5
['intense', 'intriguing', 'engaging', 'unpredictable', 'uplifting', 'engrossing', 'inspirational', 'rewarding', 'absorbing', 'informative', 'upbeat', 'enthralling', 'enlightening']

Cluster 6
['backwards', 'upstairs', 'downstairs']

Cluster 7
['lincoln', 'santiago', 'corbett']

Cluster 8
['passionate', 'sensual', 'fragile', 'seductive', 'alluring', 'dignified', 'graceful', 'counterpoint', 'earthy', 'unassuming']

Cluster 9
['palette', 'photographic', 'utilizing', 'filters', 'sepia', 'hues']


In [122]:
def create_bag_of_centroids(wordlist, word_centroid_map):
  num_centroids = max(word_centroid_map.values()) +1

  bag_of_centroids = np.zeros(num_centroids, dtype="float32")
  
  for word in wordlist:
    if word in word_centroid_map:
      index = word_centroid_map[word]
      bag_of_centroids[index] += 1
  return bag_of_centroids

In [124]:
# speed 위해 bags of centroids training set pre-allocate
train_centroids = np.zeros((train["review"].size, 
                            num_clusters),
                           dtype="float32")

# bags of centroids로 training set reviews를 변환
counter = 0
for review in clean_train_reviews:
  train_centroids[counter] = create_bag_of_centroids(review,
                                                     word_centroid_map)
  counter += 1

# repeat test reviews
test_centroids = np.zeros((test["review"].size,
                           num_clusters),
                          dtype="float32")
counter = 0
for review in clean_test_reviews:
  test_centroids[counter] = create_bag_of_centroids(review,
                                                    word_centroid_map)
  counter += 1

In [125]:
# fit a random forest and extract predictions
forest = RandomForestClassifier(n_estimators=100)

# fitting the forest 
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids, train["sentiment"])
result = forest.predict(test_centroids)

output = pd.DataFrame(data={"id": test["id"],
                            "sentiment":result})
output.to_csv("/content/drive/MyDrive/kaggle/BagsOfPopcorn/BagOfCentroids.csv", index=False, quoting=3)

Fitting a random forest to labeled training data...
